In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import time
import random

In [ ]:
def make_urllist_sub(index, date, page):
    sub_urllist = []
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(index)+'&date='+str(date)+'&page='+str(page)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)
    soup = BeautifulSoup(news.content, 'html.parser')
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
    for line in news_list:
        sub_urllist.append(line.a.get('href'))
    return sub_urllist

In [ ]:
def make_urllist(index, date):
    total_url_list = []
    temp_url = []
    for page in range(1, 1000):
        url_list = make_urllist_sub(index, date, page)
        if url_list[-1] == temp_url:
            break
        temp_url = url_list[-1]
        total_url_list.extend(url_list)
    return total_url_list

In [ ]:
def make_data(urllist):
    text_list = []
    for url in urllist:
        article = Article(url, language='ko')
        article.download()
        article.parse()
        text_list.append(article.text)
    df = pd.DataFrame({'contents': text_list})
    return df

In [ ]:
def make_total_data(index, date):
    df = None
    temp_url = []
    for date in range(date, date+7): # 기본 설정은 1주일치로 되어있습니다.
        url_list = make_urllist(index, date)
        if temp_url == url_list[-1]:
            break
        temp_url = url_list[-1]
        df_temp = make_data(url_list)
        if df is not None:
            df = pd.concat([df, df_temp])
        else:
            df = df_temp
        df.to_csv(str(date) + '.csv') # 1일치가 끝날때마다 '해당 날짜.csv'로 저장됩니다.
        time.sleep(random.uniform(5, 10)) # 1일치가 끝나면 5초에서 10초동안 쉬도록 설정되어있습니다.
    df.to_csv(str(date) + '-' + str(date+7) + '.csv') #1주일치가 끝나면 '첫날-마지막날.csv'로 저장됩니다.
    return df

In [ ]:
index = 100 #인덱스 설정은 개인별로 바꾸시길 바랍니다.

## index

### 100 : 정치
### 101 : 경제
### 102 : 사회
### 103 : 생활/문화
### 104 : 세계
### 105 : IT/과학
### 110 : 오피니

In [ ]:
startday = 20200506 #2020년 05월 06일

In [ ]:
df = make_total_data(index, startday)
df